In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import missingno as msno

from sklearn.ensemble import VotingClassifier
from sklearn.metrics import f1_score
# Boosting Algorithms 
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
%matplotlib inline

train=pd.read_csv(r'C:\Users\tanuj\Downloads\train_hr.csv')
test=pd.read_csv(r'C:\Users\tanuj\Downloads\test_hr.csv')


train.head()

,employee_id,department,region,education,gender,recruitment_channel,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,avg_training_score,is_promoted
0,65438,Sales & Marketing,region_7,Master's & above,f,sourcing,1,35,5.0,8,1,0,49,0
1,65141,Operations,region_22,Bachelor's,m,other,1,30,5.0,4,0,0,60,0
2,7513,Sales & Marketing,region_19,Bachelor's,m,sourcing,1,34,3.0,7,0,0,50,0
3,2542,Sales & Marketing,region_23,Bachelor's,m,other,2,39,1.0,10,0,0,50,0
4,48945,Technology,region_26,Bachelor's,m,other,1,45,3.0,2,0,0,73,0


In [2]:

train['previous_year_rating'].mean()
train['previous_year_rating'].var()



1.5875812309461046

# feature engineering

In [3]:
train['previous_year_rating']=train['previous_year_rating'].replace(np.nan,0)
train['education']=train['education'].replace(np.nan,(train['education'].mode())[0])

test['previous_year_rating']=test['previous_year_rating'].replace(np.nan,0)
test['education']=test['education'].replace(np.nan,(test['education'].mode())[0])


In [4]:
from sklearn.preprocessing import KBinsDiscretizer

#previous_year_rating discretizer
discretizer=KBinsDiscretizer(encode='ordinal')
train['previous_year_rating']=discretizer.fit_transform(train[['previous_year_rating']])


discretizer2=KBinsDiscretizer(encode='ordinal')
test['previous_year_rating']=discretizer.fit_transform(test[['previous_year_rating']])




#avg training score discretizer
discretizer=KBinsDiscretizer(encode='ordinal',strategy='uniform')
train['avg_training_score']=discretizer.fit_transform(train[['avg_training_score']])


discretizer2=KBinsDiscretizer(encode='ordinal',strategy='uniform')
test['avg_training_score']=discretizer.fit_transform(test[['avg_training_score']])



C:\Users\tanuj\anaconda3\lib\site-packages\sklearn\preprocessing\_discretization.py:222: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 0 are removed. Consider decreasing the number of bins.
  'decreasing the number of bins.' % jj)
C:\Users\tanuj\anaconda3\lib\site-packages\sklearn\preprocessing\_discretization.py:222: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 0 are removed. Consider decreasing the number of bins.
  'decreasing the number of bins.' % jj)


In [5]:
from sklearn.preprocessing import KBinsDiscretizer
discretizer=KBinsDiscretizer(encode='ordinal')
train['age']=discretizer.fit_transform(train[['age']])
discretizer2=KBinsDiscretizer(encode='ordinal')
test['age']=discretizer.transform(test[['age']])



train=train.drop(['region'],axis=1)
test=test.drop(['region'],axis=1)


In [6]:
x=train.drop(['employee_id','is_promoted'],axis=1)
y=train['is_promoted']
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test =train_test_split(x,y,test_size=0.25)


In [7]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
ohe=OneHotEncoder(sparse=False)



In [8]:
train.head()

,employee_id,department,education,gender,recruitment_channel,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,avg_training_score,is_promoted
0,65438,Sales & Marketing,Master's & above,f,sourcing,1,3.0,2.0,8,1,0,0.0,0
1,65141,Operations,Bachelor's,m,other,1,1.0,2.0,4,0,0,1.0,0
2,7513,Sales & Marketing,Bachelor's,m,sourcing,1,2.0,2.0,7,0,0,0.0,0
3,2542,Sales & Marketing,Bachelor's,m,other,2,3.0,0.0,10,0,0,0.0,0
4,48945,Technology,Bachelor's,m,other,1,4.0,2.0,2,0,0,2.0,0


In [9]:
# one hot encoding train
ohe_train=pd.DataFrame(ohe.fit_transform(X_train[['department','gender','recruitment_channel']]))
ohe_test=pd.DataFrame(ohe.transform(X_test[['department','gender','recruitment_channel']]))

X_train.reset_index(drop=True,inplace=True)
X_test.reset_index(drop=True,inplace=True)

X_train=pd.concat([X_train,ohe_train],axis=1)
X_test=pd.concat([X_test,ohe_test],axis=1)


In [10]:
# one hot encoding test
ohe_Test=pd.DataFrame(ohe.fit_transform(test[['department','gender','recruitment_channel']]))
test=pd.concat([test,ohe_Test],axis=1)



In [11]:
#ordinal encoder
oe=OrdinalEncoder(categories=[["Below Secondary","Bachelor's","Master's & above"]])


oe_train=pd.DataFrame(oe.fit_transform(X_train[['education']]),columns=['ed'])
oe_test=pd.DataFrame(oe.transform(X_test[['education']]),columns=['ed'])


#concating
X_train=pd.concat([X_train,oe_train],axis=1)
X_test=pd.concat([X_test,oe_test],axis=1)

In [12]:
oe_Test=pd.DataFrame(oe.fit_transform(test[['education']]),columns=['ed'])
test=pd.concat([test,oe_Test],axis=1)

In [13]:
X_train=X_train.drop(['department','education','gender','recruitment_channel'],axis=1)
X_test=X_test.drop(['department','education','gender','recruitment_channel'],axis=1)


test=test.drop(['department','education','gender','recruitment_channel'],axis=1)



In [14]:
from sklearn.tree import DecisionTreeClassifier
dt=DecisionTreeClassifier()
dt.fit(X_train,y_train)

dt.score(X_test,y_test)

0.9078236753758575

In [15]:
y_pred=dt.predict(test.iloc[:,1:])
promotion=pd.DataFrame({'employee_id':test['employee_id'],'is_promoted':y_pred})
promotion.to_csv('promotion.csv',index=False)






In [16]:
cb=CatBoostClassifier(learning_rate=0.15,
                                                 n_estimators=494,
                                                 subsample=0.085,
                                                 max_depth=5,
                                                 scale_pos_weight=2.5)
cb.fit(X_train,y_train)
cb.score(X_test,y_test)


0:	learn: 0.5605542	total: 145ms	remaining: 1m 11s
1:	learn: 0.4976969	total: 150ms	remaining: 36.8s
2:	learn: 0.4554528	total: 154ms	remaining: 25.2s
3:	learn: 0.4327617	total: 158ms	remaining: 19.4s
4:	learn: 0.4023673	total: 163ms	remaining: 15.9s
5:	learn: 0.3897345	total: 167ms	remaining: 13.6s
6:	learn: 0.3629893	total: 172ms	remaining: 11.9s
7:	learn: 0.3576388	total: 176ms	remaining: 10.7s
8:	learn: 0.3533053	total: 180ms	remaining: 9.72s
9:	learn: 0.3488974	total: 185ms	remaining: 8.94s
10:	learn: 0.3429453	total: 190ms	remaining: 8.32s
11:	learn: 0.3370675	total: 194ms	remaining: 7.79s
12:	learn: 0.3356529	total: 198ms	remaining: 7.34s
13:	learn: 0.3340267	total: 203ms	remaining: 6.95s
14:	learn: 0.3327054	total: 207ms	remaining: 6.61s
15:	learn: 0.3318426	total: 211ms	remaining: 6.31s
16:	learn: 0.3266944	total: 215ms	remaining: 6.04s
17:	learn: 0.3253219	total: 220ms	remaining: 5.82s
18:	learn: 0.3246005	total: 224ms	remaining: 5.61s
19:	learn: 0.3236717	total: 228ms	remain

0.922566048752007

In [17]:
xg=XGBClassifier()
xg.fit(X_train,y_train)
xg.score(X_test,y_test)

C:\Users\tanuj\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[01:10:24] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


0.9287695226974164

In [18]:
lg= LGBMClassifier(learning_rate =0.1,
                                           n_estimators=494,
                                           max_depth=5,
                                           subsample = 0.70,
                                           verbosity = 0,
                                           scale_pos_weight = 2.5,
                                           updater ="grow_histmaker",
                                           base_score  = 0.2)
lg.fit(X_train,y_train)
lg.score(X_test,y_test)



[LightGBM] [Warning] Unknown parameter: updater
[LightGBM] [Warning] Unknown parameter: base_score
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Unknown parameter: updater
[LightGBM] [Warning] Unknown parameter: base_score
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001930 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGB

0.9195737848489272

In [19]:
y_pred1=xg.predict(test.iloc[:,1:])
promotion1=pd.DataFrame({'employee_id':test['employee_id'],'is_promoted':y_pred1})
promotion1.to_csv('promotionger1.csv',index=False)



In [20]:
from sklearn.linear_model import LogisticRegression
lr=LogisticRegression()
lr.fit(X_train,y_train)
lr.score(X_test,y_test)


from sklearn.ensemble import RandomForestClassifier
rf=RandomForestClassifier()
lr.fit(X_train,y_train)
lr.score(X_test,y_test)

C:\Users\tanuj\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
C:\Users\tanuj\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-reg

0.9232958692161728

In [22]:
y_pred=cb.predict(test.iloc[:,1:])
promotion=pd.DataFrame({'employee_id':test['employee_id'],'is_promoted':y_pred})
promotion.to_csv('promotionscb.csv',index=False)
